In [2]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [3]:
# 식당 데이터 임포트
#식당 데이터는 드라이브 - 1.전처리파일 - 2(1)에 구별로 업로드 되어있음
name_data = pd.read_csv('마포구100.csv', encoding='utf-8-sig')
print(name_data)

#필요할경우 컬럼명 변경
# name_data.rename(columns={'Unnamed: 0':'name'}, inplace=True)
# name_data

# !식당명에 지역명이 포함되었는지 반드시 확인할 것! #

        name  restaurant
0        비터스         171
1     무쇠김치삼겹         131
2         랫댓         114
3      남산호랭이         102
4      하이디라오         100
..       ...         ...
97       크래커          11
98     닝교초식당          11
99        고미          11
100    딩가케이크          11
101    모멘트커피          11

[102 rows x 2 columns]


In [4]:
## 지역명이 포함되지 않았을 경우 지역명 추가하는 코드 ##
# 필요없는 경우 스킵 #
tmp = []
for i in name_data['name']:
    region = '마포'+i
    tmp.append(
        region
    )
name_data['name'] = tmp
name_data

,name,restaurant
0,마포 비터스,171
1,마포 무쇠김치삼겹,131
2,마포 랫댓,114
3,마포 남산호랭이,102
4,마포 하이디라오,100
...,...,...
97,마포 크래커,11
98,마포 닝교초식당,11
99,마포 고미,11
100,마포 딩가케이크,11


In [5]:
#지역명이 포함된 식당명을 변수로 지정
items = name_data['name']
print(items)

0         마포 비터스
1      마포 무쇠김치삼겹
2          마포 랫댓
3       마포 남산호랭이
4       마포 하이디라오
         ...    
97        마포 크래커
98      마포 닝교초식당
99         마포 고미
100     마포 딩가케이크
101     마포 모멘트커피
Name: name, Length: 102, dtype: object


In [6]:
#검색할 식당 데이터와 url을 담을 데이터 프레임 생성

df = pd.DataFrame(columns=['name', 'naverURL'])

#데이터 프레임이 잘 만들어졌는지 확인
df['name'] = items
df

,name,naverURL
0,마포 비터스,NaN
1,마포 무쇠김치삼겹,NaN
2,마포 랫댓,NaN
3,마포 남산호랭이,NaN
4,마포 하이디라오,NaN
...,...,...
97,마포 크래커,NaN
98,마포 닝교초식당,NaN
99,마포 고미,NaN
100,마포 딩가케이크,NaN


In [7]:
# 식당 url 얻기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup

for i, keyword in enumerate(df['name'].tolist()):
    # 검색 url 만들기
    naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'  
    # 검색 url 접속 = 검색하기
    driver.get(naver_map_search_url)
    time.sleep(2) 
    # 검색 프레임 변경
    driver.switch_to.frame("searchIframe")
    time.sleep(1) 
     
    
    try:     
        #식당 정보가 있다면 첫번째 식당의 url을 가져오기
        
        if len(driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li')) != 0:   
            #식당 정보 클릭        
            driver.execute_script('return document.querySelector("#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.ouxiq > a:nth-child(1) > div").click()')
            time.sleep(2)
            
            # 검색한 플레이스의 개별 페이지 저장
            tmp = driver.current_url  
            res_code = re.findall(r"place/(\d+)", tmp)
            final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
        
            print(final_url)
            df['naverURL'][i]=final_url 
        
    except: 
        df['naverURL'][i]= ''
        print('none') 
    
driver.close()



https://pcmap.place.naver.com/restaurant/1595745527/review/visitor#
none
https://pcmap.place.naver.com/restaurant/1849652804/review/visitor#
https://pcmap.place.naver.com/restaurant/1726349203/review/visitor#
none
https://pcmap.place.naver.com/restaurant/1612138237/review/visitor#
https://pcmap.place.naver.com/restaurant/687355720/review/visitor#
https://pcmap.place.naver.com/restaurant/1768254146/review/visitor#
https://pcmap.place.naver.com/restaurant/1440779896/review/visitor#
https://pcmap.place.naver.com/restaurant/1165474756/review/visitor#
https://pcmap.place.naver.com/restaurant/1771873244/review/visitor#
https://pcmap.place.naver.com/restaurant/364996702/review/visitor#
https://pcmap.place.naver.com/restaurant/37624477/review/visitor#
https://pcmap.place.naver.com/restaurant/1818853219/review/visitor#
https://pcmap.place.naver.com/restaurant/86471200/review/visitor#
https://pcmap.place.naver.com/restaurant/1844126724/review/visitor#
https://pcmap.place.naver.com/restaurant/110

In [8]:
#식당명과 url이 잘 얻어져왔는지 확인하기
print(df)


          name                                           naverURL
0       마포 비터스  https://pcmap.place.naver.com/restaurant/15957...
1    마포 무쇠김치삼겹                                                   
2        마포 랫댓  https://pcmap.place.naver.com/restaurant/18496...
3     마포 남산호랭이  https://pcmap.place.naver.com/restaurant/17263...
4     마포 하이디라오                                                   
..         ...                                                ...
97      마포 크래커  https://pcmap.place.naver.com/restaurant/37569...
98    마포 닝교초식당  https://pcmap.place.naver.com/restaurant/10294...
99       마포 고미  https://pcmap.place.naver.com/restaurant/11954...
100   마포 딩가케이크  https://pcmap.place.naver.com/restaurant/58999...
101   마포 모멘트커피  https://pcmap.place.naver.com/restaurant/10996...

[102 rows x 2 columns]


In [9]:
#url을 얻어오지 못한 식당 확인
print(df.loc[df['naverURL']==''])
## 식당 리뷰 url이 들어가있지 않은 경우 직접 검색하여 데이터에 넣어주기!!! ##

         name naverURL
1   마포 무쇠김치삼겹         
4    마포 하이디라오         
18      마포 갓잇         
29      마포 솔솥         
40     마포 미도인         
43    마포 감성타코         
51     마포 온달집         
91     마포 토끼정         


In [10]:
# url을 얻어오지 못한 식당의 리뷰 url 추가 #

#네이버지도에 식당 검색 후 뜨는 map.naver.com/place/고유번호
#고유번호를  'https://pcmap.place.naver.com/restaurant/고유번호/review/visitor# 형식으로 추가해줄 것
#url을 추가해야할 인덱스 번호는 위 코드에서 확인 가능
#df['naverURL'][인덱스 번호]='https://pcmap.place.naver.com/restaurant/고유번호/review/visitor#' 형식
 
# url을 얻어오지 못한 식당의 url 추가
df['naverURL'][1]='https://pcmap.place.naver.com/restaurant/1609583532/review/visitor#'
df['naverURL'][4]='https://pcmap.place.naver.com/restaurant/653467130/review/visitor#'
df['naverURL'][18]='https://pcmap.place.naver.com/restaurant/1690354455/review/visitor#'
df['naverURL'][29]='https://pcmap.place.naver.com/restaurant/1698847103/review/visitor#'
df['naverURL'][40]='https://pcmap.place.naver.com/restaurant/1436718221/review/visitor#'
df['naverURL'][43]='https://pcmap.place.naver.com/restaurant/625311806/review/visitor#'
df['naverURL'][51]='https://pcmap.place.naver.com/restaurant/1292304113/review/visitor#'
df['naverURL'][91]='https://pcmap.place.naver.com/restaurant/38314477/review/visitor#'

#데이터 확인
print(df.loc[df['naverURL']==''])

Empty DataFrame
Columns: [name, naverURL]
Index: []


In [42]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(df['name']) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for i in range(len(df)): 
    
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naverURL'][i]) 
    thisurl = df['naverURL'][i]
    time.sleep(2)
    print('현재 수집중인 식당 : ', df['name'][i])
    
    #리뷰 더보기 버튼 다 누르기
    while True: 
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(3) 
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').click() 
            
            time.sleep(2) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(1) 
            
        except NoSuchElementException: 
            print('-모든 리뷰 더보기 완료-') 
            break 
        
    #식당 평균 별점 수집
    try:
        rating = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div.place_section_content > div > div.Xj_yJ > span.m7jAR.ohonc > em').text
        print('식당 평균 별점 : ', rating)
        rev_list.append(
            [df['name'][i],
             rating
             ]
        )
    except:
        pass
    
    

     
        
    #리뷰 데이터 스크래핑을 위한 html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        #키워드 리뷰가 아닌 리뷰글 리스트 검색
        review_lists = soup.select('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div > ul > li')
        print('총 리뷰 수 : ', len(review_lists))
        
        #리뷰 수가 0이 아닌 경우 리뷰 수집
        if len(review_lists) > 0 :
            
            for j, review in enumerate(review_lists):
                
                try:
                    #리뷰 내용이 있는 경우 리뷰 수집
                    driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div > ul > li > div.ZZ4OK.IwhtZ')
                    
                    #내용 더보기가 있는 경우 내용 더보기를 눌러주기
                    try:
                        driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div > ul > li > div.ZZ4OK.IwhtZ > a > span.rvCSr > svg').click()
                        time.sleep(1)
   
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                        
                        
                                
                
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        if len(user_review) > 0:
                             rev_list.append(
                                [
                                    df['name'][i],
                                    '',
                                    user_review[0].text
                                ]
                                )      
                       
                        time.sleep(2)            
                
                    
                    
                    except NoSuchElementException:
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                                
                        
                        #식당 평균 평점
                        #rating = driver.find_element(by='xpath', value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em')
                        
        
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        if len(user_review) > 0:
                            rev_list.append(
                                [
                                    df['name'][i],
                                    '',
                                    user_review[0].text
                                ]
                                )      
                        
                        time.sleep(2) 
                        
                except NoSuchElementException:
                    print('선택자를 찾지 못함')
                    continue  
                
        else:
            print('리뷰가 없습니다')
            time.sleep(1)                
        
       
        


    # 리뷰가 없는 경우        
    except NoSuchElementException: 
               
        rev_list.append(
        [
            df['name'][i],
            rating,
        ]
        ) 
        time.sleep(2)           
        print("리뷰가 없습니다")

    
            
    #검색한 창 닫고 검색 페이지로 돌아가기    
    # driver.close()
    # driver.switch_to.window(tabs[0])
    print("기본 페이지로 돌아가기")

        
driver.close()

진행상황 : 1/4
현재 수집중인 식당 :  만년곱창
-모든 리뷰 더보기 완료-
총 리뷰 수 :  2
기본 페이지로 돌아가기
진행상황 : 2/4
현재 수집중인 식당 :  홍리마라탕
-모든 리뷰 더보기 완료-
총 리뷰 수 :  4
기본 페이지로 돌아가기
진행상황 : 3/4
현재 수집중인 식당 :  히글리
-모든 리뷰 더보기 완료-
총 리뷰 수 :  22
기본 페이지로 돌아가기
진행상황 : 4/4
현재 수집중인 식당 :  수원옛통닭
-모든 리뷰 더보기 완료-
식당 평균 별점 :  4.31
총 리뷰 수 :  8
기본 페이지로 돌아가기


In [43]:
#스크래핑한 데이터를 데이터 프레임으로 만들기  
column = ["name", 'rate', "review"]
df2 = pd.DataFrame(rev_list, columns=column)
df2   

ValueError: 4 columns passed, passed data had 3 columns

In [ ]:
#스크래핑한 데이터를 csv 파일로 저장
df2.to_csv('마포구_네이버리뷰.csv', encoding='utf-8-sig')